Example
Let's say we have a complex strategy that has lots of (hyper-)parameters that have to be tuned. While brute-forcing all combinations seems to be a rather unrealistic attempt, we can still interpolate, and vectorbt makes exactly this possible. It doesn't care whether we have one strategy instance or millions. As soon as their vectors can be concatenated into a matrix and we have enough memory, we can analyze them in one go.

Let's start with fetching the daily price of Bitcoin:

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import vectorbt as vbt

In [3]:
start = '2019-01-01 UTC'  # crypto is in UTC
end = '2020-01-01 UTC'

In [4]:
btc_price = vbt.YFData.download('BTC-USD', start=start, end=end).get('Close')

We are going to test a simple Dual Moving Average Crossover (DMAC) strategy. For this, we are going to use MA class for calculating moving averages and generating signals.

Our first test is rather simple: buy when the 10-day moving average crosses above the 20-day moving average, and sell when opposite.

In [5]:
fast_ma = vbt.MA.run(btc_price, 10, short_name='fast')
slow_ma = vbt.MA.run(btc_price, 20, short_name='slow')

In [6]:
entries = fast_ma.ma_crossed_above(slow_ma)

In [7]:
exits = fast_ma.ma_crossed_below(slow_ma)

In [8]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

In [9]:
pf.total_return()

0.6351860771192923

One strategy instance of DMAC produced one column in signals and one performance value.

Adding one more strategy instance is as simple as adding one more column. Here we are passing an array of window sizes instead of a single value. For each window size in this array, it computes a moving average over the entire price series and stores it in a distinct column.

In [10]:
# Multiple strategy instances: (10, 30) and (20, 30)
fast_ma = vbt.MA.run(btc_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(btc_price, [30, 30], short_name='slow')

In [11]:
entries = fast_ma.ma_crossed_above(slow_ma)

In [12]:
exits = fast_ma.ma_crossed_below(slow_ma)

In [13]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.total_return()

fast_window  slow_window
10           30             0.847151
20           30             0.543411
Name: total_return, dtype: float64

For the sake of convenience, vectorbt has created the column levels fast_window and slow_window for us to easily distinguish which window size corresponds to which column.

Notice how signal generation part remains the same for each example - most functions in vectorbt work on time series of any shape. This allows creation of analysis pipelines that are universal to input data.

The representation of different features as columns offers endless possibilities for backtesting. We could, for example, go a step further and conduct the same tests for Ethereum. To compare both instruments, combine price series for Bitcoin and Ethereum into one DataFrame and run the same backtesting pipeline.

In [14]:
# Multiple strategy instances and instruments
eth_price = vbt.YFData.download('ETH-USD', start=start, end=end).get('Close')
comb_price = btc_price.vbt.concat(eth_price,
    keys=pd.Index(['BTC', 'ETH'], name='symbol'))
comb_price.vbt.drop_levels(-1, inplace=True)

comb_price

symbol,BTC,ETH
Date,,
2019-01-01 00:00:00+00:00,3843.520020,140.819412
2019-01-02 00:00:00+00:00,3943.409424,155.047684
2019-01-03 00:00:00+00:00,3836.741211,149.135010
2019-01-04 00:00:00+00:00,3857.717529,154.581940
2019-01-05 00:00:00+00:00,3845.194580,155.638596
...,...,...
2019-12-27 00:00:00+00:00,7290.088379,127.214607
2019-12-28 00:00:00+00:00,7317.990234,128.322708
2019-12-29 00:00:00+00:00,7422.652832,134.757980


In [15]:
fast_ma = vbt.MA.run(comb_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(comb_price, [30, 30], short_name='slow')

In [16]:
entries = fast_ma.ma_crossed_above(slow_ma)

In [17]:
exits = fast_ma.ma_crossed_below(slow_ma)

In [18]:
pf = vbt.Portfolio.from_signals(comb_price, entries, exits)
pf.total_return()

fast_window  slow_window  symbol
10           30           BTC       0.847151
                          ETH       0.244204
20           30           BTC       0.543411
                          ETH      -0.319102
Name: total_return, dtype: float64

In [19]:
mean_return = pf.total_return().groupby('symbol').mean()
mean_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Mean total return')

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': 'e4033c48-7fd8-4008-91f3-4f3df589b463',
              'x': array(['BTC', 'ETH'], dtype=object),
              'y': array([ 0.69528142, -0.03744928])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700,
               'xaxis': {'title': {'text': 'Symbol'}},
               'yaxis': {'title': {'text': 'Mean total return'}}}
})